# CROP Arima model
This notebook checks outputs of the Arima model

In [23]:
#!pip3 install psycopg2
#!pip3 install plotly

In [24]:
import os
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Parameters

In [25]:
crop_host = "cropapptestsqlserver.postgres.database.azure.com"
crop_port = "5432"
crop_dbname = "app_db"
crop_user = "cropdbadmin@cropapptestsqlserver"
crop_password = "QhXZ7qZddDr224Mc2P4k"

In [26]:
conn = psycopg2.connect(
    host=crop_host, port=crop_port, dbname=crop_dbname, 
    user=crop_user, password=crop_password)

cur = conn.cursor()

### Arima Data Access

All Arima data for 3 sensors

In [27]:
#dt_to = datetime.now()
dt_to = datetime(2021, 4, 27)  
dt_from = dt_to + timedelta(days=-45)

In [28]:
#sql_command = """SELECT * FROM model_run"""

sql_command = """SELECT public.model.id as model_id, public.model_run.id as modelrun_id, public.model_product.run_id as run_id, public.model_product.id as product_id, public.model_run.sensor_id, public.model_value.prediction_value as p_value, public.model_value.prediction_index as p_index, public.model_measure.measure_name, public.model_run.time_forecast as forecast
  FROM public.model_measure
LEFT OUTER JOIN public.model_product ON public.model_measure.id = public.model_product.measure_id
LEFT OUTER JOIN public.model_run ON public.model_product.run_id = public.model_run.id
LEFT OUTER JOIN public.model ON public.model_run.model_id = public.model.id
LEFT OUTER JOIN public.model_value ON public.model_product.id = public.model_value.product_id
WHERE (public.model.id = 1)
ORDER BY public.model_product.id, public.model_value.prediction_index
"""

In [29]:
sql_command

'SELECT public.model.id as model_id, public.model_run.id as modelrun_id, public.model_product.run_id as run_id, public.model_product.id as product_id, public.model_run.sensor_id, public.model_value.prediction_value as p_value, public.model_value.prediction_index as p_index, public.model_measure.measure_name, public.model_run.time_forecast as forecast\n  FROM public.model_measure\nLEFT OUTER JOIN public.model_product ON public.model_measure.id = public.model_product.measure_id\nLEFT OUTER JOIN public.model_run ON public.model_product.run_id = public.model_run.id\nLEFT OUTER JOIN public.model ON public.model_run.model_id = public.model.id\nLEFT OUTER JOIN public.model_value ON public.model_product.id = public.model_value.product_id\nWHERE (public.model.id = 1)\nORDER BY public.model_product.id, public.model_value.prediction_index\n'

In [30]:
arima_raw = pd.read_sql(sql_command, conn)
arima = arima_raw.drop_duplicates()
arima

,model_id,modelrun_id,run_id,product_id,sensor_id,p_value,p_index,measure_name,forecast
0,1,80,80,99,18,22.802063,1,Mean Temperature (Degree Celcius),2021-06-14 04:00:00
1,1,80,80,99,18,22.853876,2,Mean Temperature (Degree Celcius),2021-06-14 04:00:00
2,1,80,80,99,18,22.662361,3,Mean Temperature (Degree Celcius),2021-06-14 04:00:00
3,1,80,80,99,18,22.052952,4,Mean Temperature (Degree Celcius),2021-06-14 04:00:00
4,1,80,80,99,18,22.097458,5,Mean Temperature (Degree Celcius),2021-06-14 04:00:00
...,...,...,...,...,...,...,...,...,...
6187,1,133,133,252,23,21.310948,44,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00
6188,1,133,133,252,23,22.078962,45,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00
6189,1,133,133,252,23,21.330075,46,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00
6190,1,133,133,252,23,21.528400,47,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00


## Calculate and add column with prediciton time

In [31]:
prediction_hours= []

for i in range(len(arima)):
   prediction_hours.append(arima['forecast'][i] + timedelta(hours= int(arima['p_index'][i])))

arima['prediction_time'] = prediction_hours

arima


,model_id,modelrun_id,run_id,product_id,sensor_id,p_value,p_index,measure_name,forecast,prediction_time
0,1,80,80,99,18,22.802063,1,Mean Temperature (Degree Celcius),2021-06-14 04:00:00,2021-06-14 05:00:00
1,1,80,80,99,18,22.853876,2,Mean Temperature (Degree Celcius),2021-06-14 04:00:00,2021-06-14 06:00:00
2,1,80,80,99,18,22.662361,3,Mean Temperature (Degree Celcius),2021-06-14 04:00:00,2021-06-14 07:00:00
3,1,80,80,99,18,22.052952,4,Mean Temperature (Degree Celcius),2021-06-14 04:00:00,2021-06-14 08:00:00
4,1,80,80,99,18,22.097458,5,Mean Temperature (Degree Celcius),2021-06-14 04:00:00,2021-06-14 09:00:00
...,...,...,...,...,...,...,...,...,...,...
6187,1,133,133,252,23,21.310948,44,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00,2021-11-06 00:00:00
6188,1,133,133,252,23,22.078962,45,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00,2021-11-06 01:00:00
6189,1,133,133,252,23,21.330075,46,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00,2021-11-06 02:00:00
6190,1,133,133,252,23,21.528400,47,Lower Bound Temperature (Degree Celcius),2021-11-04 04:00:00,2021-11-06 03:00:00


In [32]:
arima = arima[arima['run_id'] == 82]
arima.describe()


,model_id,modelrun_id,run_id,product_id,sensor_id,p_value,p_index
count,144.0,144.0,144.0,144.000000,144.0,144.000000,144.000000
mean,1.0,82.0,82.0,106.000000,23.0,22.575537,24.500000
std,0.0,0.0,0.0,0.819346,0.0,2.780740,13.901753
min,1.0,82.0,82.0,105.000000,23.0,17.444805,1.000000
25%,1.0,82.0,82.0,105.000000,23.0,19.733789,12.750000
50%,1.0,82.0,82.0,106.000000,23.0,22.729274,24.500000
75%,1.0,82.0,82.0,107.000000,23.0,25.208721,36.250000
max,1.0,82.0,82.0,107.000000,23.0,27.087229,48.000000


In [33]:
#unique_run_ids = arima['run_id'].unique()
unique_sensors = arima['sensor_id'].unique()
unique_time_forecast = arima['forecast'].unique()
unique_measures = arima['measure_name'].unique()

unique_time_forecast

array(['2021-06-14T04:00:00.000000000'], dtype='datetime64[ns]')

In [34]:
date_time = pd.to_datetime(unique_time_forecast[0])
date_time
#type(date_time)
#type(datetime(2021, 6, 16))

Timestamp('2021-06-14 04:00:00')

### Get Temperature data from Zensie Sensors

All Zensie sensors

In [35]:
dt_to_z = date_time + timedelta(days=+3) #datetime(2021, 6, 16)
dt_from_z = dt_to_z + timedelta(days=-5)

In [36]:
sql_command_zensie = """SELECT sensors.name, zensie_trh_data.* FROM sensor_types, sensors, zensie_trh_data WHERE sensors.id = zensie_trh_data.sensor_id AND zensie_trh_data.timestamp >= '%s' AND zensie_trh_data.timestamp < '%s'""" % (dt_from_z, dt_to_z)

In [37]:
zensie_raw = pd.read_sql(sql_command_zensie, conn)
zensie_df = zensie_raw.drop_duplicates()
zensie_df

,name,id,sensor_id,timestamp,temperature,humidity,time_created,time_updated
0,FARM_T/RH_16B1,545611,18,2021-06-12 04:05:00+00:00,25.150000,77.0,2021-06-12 05:00:04.305257,None
5,FARM_T/RH_16B1,545612,18,2021-06-12 04:15:00+00:00,25.150000,77.0,2021-06-12 05:00:04.305257,None
10,FARM_T/RH_16B1,545613,18,2021-06-12 04:25:00+00:00,25.100000,77.0,2021-06-12 05:00:04.305257,None
15,FARM_T/RH_16B1,545614,18,2021-06-12 04:35:00+00:00,25.100000,77.0,2021-06-12 05:00:04.305257,None
20,TUNNEL7_T/RH_1,545617,19,2021-06-12 04:00:00+00:00,23.250000,90.0,2021-06-12 05:00:06.398971,None
...,...,...,...,...,...,...,...,...
29215,Farm_T/RH_16B4,551487,23,2021-06-17 03:55:00+00:00,25.350000,74.0,2021-06-17 05:00:16.170118,None
29220,Farm_T/RH_16B2,551492,27,2021-06-17 03:45:00+00:00,25.200001,73.0,2021-06-17 05:00:18.498218,None
29225,Farm_T/RH_16B2,551493,27,2021-06-17 03:55:00+00:00,25.200001,73.0,2021-06-17 05:00:18.498218,None
29230,R&D_T/RH_1,551498,24,2021-06-17 03:40:00+00:00,23.850000,73.0,2021-06-17 05:00:20.748200,None


## Visualisation and parameters

In [38]:
time_forecast_id= 0
time_forecast = unique_time_forecast[time_forecast_id]

In [39]:
color = ["red", 'white', 'white']
for j in range (len(unique_sensors)):
    

    # set up plotly figure
    fig = go.Figure()
    
    arima_df = arima[(arima['sensor_id'] == unique_sensors[j]) & (arima['forecast'] == time_forecast)]

    # add line / trace 1 to figure
    for i in range(len (unique_measures)):
        trace_sensor = arima_df[arima_df['measure_name'] == unique_measures[i]]
        fig.add_trace(go.Scatter(
            x=trace_sensor['prediction_time'],
            y=trace_sensor['p_value'],
            #hovertext=df['A_info'],
            #hoverinfo="text",
            name=unique_measures[i],#unique_measures[j],
            marker=dict(
                color=color[i],
            ),
            showlegend=True
        ))

    # add line / trace 2 to figure
    
    zensie_df_s = zensie_df[zensie_df['sensor_id'] == unique_sensors[j]]
    fig.add_trace(go.Scatter(
        x=zensie_df_s['timestamp'],
        y=zensie_df_s['temperature'],
        name='zensie',
        marker=dict(
            color="green"
        ),
        showlegend=False
    ))

    fig.update_layout(
    title="Arima and Zensie data: Sensor id %s, run_id %s" % (unique_sensors[j], arima_df['run_id'].unique()) ,
    xaxis_title="Date",
    yaxis_title="Temperature")

    fig.show()
    


## Calculate Square root mean error

In [40]:
import sklearn
from sklearn.metrics import mean_squared_error
import bisect

In [41]:
unique_measures

array(['Mean Temperature (Degree Celcius)',
       'Upper Bound Temperature (Degree Celcius)',
       'Lower Bound Temperature (Degree Celcius)'], dtype=object)

In [42]:
zensie_f = zensie_df[['timestamp','temperature']]


In [43]:
f= zensie_f.groupby(by=[zensie_f.timestamp.map(lambda x: "%04d-%02d-%02d-%02d" % (x.year, x.month, x.day, x.hour)), "temperature"]).sum()


                 

f

Empty DataFrame
Columns: []
Index: [(2021-06-12-04, 21.2999992370605), (2021-06-12-04, 21.3500003814697), (2021-06-12-04, 23.25), (2021-06-12-04, 23.2999992370605), (2021-06-12-04, 23.8500003814697), (2021-06-12-04, 23.8999996185303), (2021-06-12-04, 24.75), (2021-06-12-04, 25.1000003814697), (2021-06-12-04, 25.1499996185303), (2021-06-12-04, 25.7999992370605), (2021-06-12-04, 25.8500003814697), (2021-06-12-04, 25.8999996185303), (2021-06-12-04, 28.3500003814697), (2021-06-12-04, 28.3999996185303), (2021-06-12-05, 21.1499996185303), (2021-06-12-05, 21.25), (2021-06-12-05, 21.2999992370605), (2021-06-12-05, 23.2999992370605), (2021-06-12-05, 23.3999996185303), (2021-06-12-05, 23.7000007629395), (2021-06-12-05, 23.8500003814697), (2021-06-12-05, 23.8999996185303), (2021-06-12-05, 24.7000007629395), (2021-06-12-05, 24.75), (2021-06-12-05, 24.7999992370605), (2021-06-12-05, 24.8500003814697), (2021-06-12-05, 25.1000003814697), (2021-06-12-05, 25.1499996185303), (2021-06-12-05, 25.4500007629395), (2021-06-12-05, 25.5499992370605), (2021-06-12-05, 25.7999992370605), (2021-06-12-05, 25.8500003814697), (2021-06-12-05, 25.8999996185303), (2021-06-12-05, 27.7000007629395), (2021-06-12-05, 28.3500003814697), (2021-06-12-05, 28.3999996185303), (2021-06-12-06, 19.6499996185303), (2021-06-12-06, 19.8999996185303), (2021-06-12-06, 20.0), (2021-06-12-06, 20.1499996185303), (2021-06-12-06, 20.25), (2021-06-12-06, 20.4500007629395), (2021-06-12-06, 23.1499996185303), (2021-06-12-06, 23.2999992370605), (2021-06-12-06, 23.3500003814697), (2021-06-12-06, 23.3999996185303), (2021-06-12-06, 23.4500007629395), (2021-06-12-06, 23.5), (2021-06-12-06, 23.5499992370605), (2021-06-12-06, 23.6000003814697), (2021-06-12-06, 23.7000007629395), (2021-06-12-06, 23.8500003814697), (2021-06-12-06, 23.8999996185303), (2021-06-12-06, 23.9500007629395), (2021-06-12-06, 24.1499996185303), (2021-06-12-06, 24.2999992370605), (2021-06-12-06, 24.4500007629395), (2021-06-12-06, 24.5), (2021-06-12-06, 24.5499992370605), (2021-06-12-06, 24.6000003814697), (2021-06-12-06, 24.7000007629395), (2021-06-12-06, 24.75), (2021-06-12-06, 24.8500003814697), (2021-06-12-06, 25.0), (2021-06-12-06, 25.2000007629395), (2021-06-12-06, 25.3999996185303), (2021-06-12-06, 25.75), (2021-06-12-06, 25.9500007629395), (2021-06-12-06, 26.2999992370605), (2021-06-12-06, 26.8500003814697), (2021-06-12-07, 19.6000003814697), (2021-06-12-07, 19.7000007629395), (2021-06-12-07, 19.75), (2021-06-12-07, 19.7999992370605), (2021-06-12-07, 22.7000007629395), (2021-06-12-07, 22.75), (2021-06-12-07, 22.7999992370605), (2021-06-12-07, 22.8500003814697), (2021-06-12-07, 22.9500007629395), (2021-06-12-07, 23.0499992370605), (2021-06-12-07, 23.1000003814697), (2021-06-12-07, 23.1499996185303), (2021-06-12-07, 23.2000007629395), (2021-06-12-07, 23.25), (2021-06-12-07, 23.2999992370605), (2021-06-12-07, 23.3500003814697), (2021-06-12-07, 23.3999996185303), (2021-06-12-07, 23.4500007629395), (2021-06-12-07, 23.5499992370605), (2021-06-12-07, 23.6000003814697), (2021-06-12-07, 23.7000007629395), (2021-06-12-07, 23.75), (2021-06-12-07, 23.7999992370605), (2021-06-12-07, 23.8999996185303), (2021-06-12-07, 23.9500007629395), (2021-06-12-07, 24.0), (2021-06-12-07, 24.1000003814697), (2021-06-12-07, 24.5499992370605), (2021-06-12-07, 24.6000003814697), (2021-06-12-07, 24.6499996185303), ...]

[2793 rows x 0 columns]

In [44]:
myPairs = list(zensie_f.iteritems())
myPairs.sort()

import bisect
i = bisect.bisect_left(myPairs, arima['prediction_time'])


#i = bisect.bisect_left(myPairs, arima['prediction_time'][0])

ValueError: Lengths must match

In [ ]:
actual = zensie_df['temperature']
predicted = arima_df [(arima['measure_name'] == unique_measures[0])] 

mse = sklearn.metrics.mean_squared_error(actual, predicted)

rmse = math.sqrt(mse)